In [4]:
import pandas as pd
import pitcher

In [5]:
#currently these functions will only grab pitches CU, FF, SL, and CH so only use verlander and scherzer datasets
player = 'scherzer'
data = pd.read_csv('Data/raw_data/'+player+'.csv')
data = pitcher.clean_data(data)
games = pitcher.get_games(data)
reps = pitcher.get_reps(games)
reps = pitcher.drop_nas(reps)
reps = pitcher.drop_pitches(reps)

/home/zac/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
data = []
for rep in reps:
    row = []
    prev_pitches,prev_types,pre_pitch,ptype = rep
    prevs = []
    for prev in prev_pitches:
        prevs += prev
    row += prevs + pre_pitch + [ptype]
    data.append(row)

In [11]:
len(data[0])

141

running the cell above to get 'data' gives a list of rows each of length 141, which represents the five length 25 vectors plus the one length 15 vector plus the pitch type, i.e. the inputs to the network.